In [5]:
import requests
import pandas as pd
from io import StringIO
import json
import unicodedata
import numpy as np
import nltk
import predict_flair
from flair.models import SequenceTagger
import fr_core_news_lg as fr
from python_heideltime import Heideltime
import ipinfo
import urllib.request
from ip2geotools.databases.noncommercial import DbIpCity
import re

In [6]:
communes = pd.read_csv("Data/communes_names_distinct_.csv").to_numpy().reshape(-1)

In [7]:
import difflib
import fasttext.util

fasttext.util.download_model('fr', if_exists='ignore') 
ft = fasttext.load_model('cc.fr.300.bin')

#fasttext Embeddings

df = pd.read_csv("Data/communes_embeddings.csv")
communes = np.array(df["comm"])
vectors = np.array(df.drop('comm', axis='columns', inplace=False))
embeddings = {communes[i]: vectors[i] for i in range(len(communes)) }

In [8]:
def stem(word):
    word_ = "".join(list(word)[-4:])
    return word[:-4]+re.sub(r'iens|ains|ards|ain|ien|ard|ois|oi|ens|en|ais|ai|ins|in|s$', '',word_, count = 1)

In [10]:
stem("parisien")

'paris'

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

def get_location_from_adj(c, communes):
#     emb = ft.get_word_vector(c)
#     dist = np.array(list(map(lambda a: nltk.edit_distance(c, a), communes)))
#     cosine_sim = np.array(list(map(lambda a: cosine_similarity([emb], [embeddings[a]])[0][0] , communes)))
    
#     sim1 = np.array(list(map(lambda a:difflib.SequenceMatcher(None, p1, "".join(list(a)[:3])).ratio(), communes)))
#     sim2 = np.array(list(map(lambda a:difflib.SequenceMatcher(None, p2, "".join(list(a)[3:])).ratio(), communes)))
#     avg_sim = 0.6*cosine_sim + 0.4*sim

#     if len(c) < 4: 
#         limit = 3
        
#     else:
#         limit = 4
   
    #lim = min(int(2*len(a)/3), len(c))
        
#     p1 = "".join(list(c)[:min(int(2*len(a)/3), len(c))])
#     p2 = "".join(list(c)[min(int(2*len(a)/3), len(c)):])


    

#     dist = np.array(list(map(lambda a:[nltk.edit_distance("".join(list(c)[:min(int(2*len(c)/3), len(a))]), "".join(list(a)[:min(int(2*len(c)/3), len(a))])), \
#                                         nltk.edit_distance("".join(list(c)[min(int(2*len(c)/3), len(a)):]), "".join(list(a)[min(int(2*len(c)/3), len(a)):]))], \
#                                         communes)))   
#     avg_ = 0.6*dist[:, 0] + 0.4*dist[:, 1]


#     dist = np.array(list(map(lambda a: nltk.edit_distance(c_, a), communes)))
#     sim = np.array(list(map(lambda a:difflib.SequenceMatcher(None, c_, a).ratio(), communes)))

#     sim = dist = np.array(list(map(lambda a:[ difflib.SequenceMatcher(None, "".join(list(c_)[:min(int(2*len(c_)/3), len(a))]), "".join(list(a)[:min(int(2*len(c_)/3), len(a))])).retio(), \
#                                               difflib.SequenceMatcher(None, "".join(list(c_)[min(int(2*len(c_)/3), len(a)):]), "".join(list(a)[min(int(2*len(c_)/3), len(a)):])).ratio()], \
#                                               communes))) 

#     dist = []
#     for a in communes :
#         limit = min(int(len(c_)/3), len(a))
#         a1, a2 =  "".join(list(a)[:limit]), "".join(list(a)[limit:])
#         c1, c2 =  "".join(list(c_)[:limit]), "".join(list(c_)[limit:])
#         d1, d2 = nltk.edit_distance(c1, a1), nltk.edit_distance(c2, a2)
#         dist.append([d1, d2])                                          
    c_ = stem(c)
    dist = []
    for a in communes :
        limit = min(int(2*len(c)/3), len(a))
        a1 =  "".join(list(a)[:limit])
        c1 =  "".join(list(c)[:limit])
        d1 = nltk.edit_distance(c1, a1)
        d2 = nltk.edit_distance(c_, a)
        dist.append([d1, d2])    
    
    dist = np.array(dist)
    avg = 0.5*dist[:, 0] + 0.5*dist[:, 1]
    sorted_ = np.argsort(avg)
    
    commune_ = communes[sorted_[0]]
    
#     emb = ft.get_word_vector(c)
#     cosine_sim = np.array(list(map(lambda a: cosine_similarity([emb], [embeddings[a]])[0][0] , communes_)))
#     sorted__ = np.argsort(cosine_sim)
    
#     return communes_[np.flip(sorted__)], communes_, avg_[np.flip(sorted__)], avg_
    return commune_
    

In [12]:
z = get_location_from_adj("orleannais", communes) #arboisien agathois

In [13]:
z

'orleans'

In [14]:
communes2 = pd.read_csv("Data/communes2.csv").to_numpy().reshape(-1)
adjs2 = pd.read_csv("Data/gentile2.csv").to_numpy().reshape(-1)

In [18]:
np.array([1 if get_location_from_adj(adjs2[i], communes) == communes2[i] else 0 for i in range(len(communes2))]).sum()

933

In [ ]:
from lexicons_builder import build_lexicon

In [368]:
output = build_lexicon(["book", "read"], 'en', 1, web=True)
output.to_list()

[2021-06-23 15:27:17,518] -[root] - [INFO] - looking up synonyms online for word 'book' at depth 1
[2021-06-23 15:27:17,520] -[root] - [INFO] - scrapping 'lexico.com' lang is 'en'
[2021-06-23 15:27:17,521] -[root] - [DEBUG] - Exploring with word 'book' current depth is '1'
[2021-06-23 15:27:18,376] -[root] - [INFO] - getting https://www.lexico.com/synonyms/book
[2021-06-23 15:27:18,378] -[urllib3.connectionpool] - [DEBUG] - Starting new HTTPS connection (1): www.lexico.com:443
[2021-06-23 15:27:18,669] -[urllib3.connectionpool] - [DEBUG] - https://www.lexico.com:443 "GET /synonyms/book HTTP/1.1" 200 None
[2021-06-23 15:27:18,731] -[root] - [INFO] - 22 found
[2021-06-23 15:27:18,739] -[root] - [DEBUG] - Exploring with word 'notepad' current depth is '2'
[2021-06-23 15:27:18,753] -[root] - [DEBUG] - Exploring with word 'novel' current depth is '2'
[2021-06-23 15:27:18,768] -[root] - [DEBUG] - Exploring with word 'reserve' current depth is '2'
[2021-06-23 15:27:18,782] -[root] - [DEBUG] -

[2021-06-23 15:27:21,438] -[root] - [DEBUG] - Exploring with word 'appropriate' current depth is '2'
[2021-06-23 15:27:21,469] -[root] - [DEBUG] - Exploring with word 'hold in' current depth is '2'
[2021-06-23 15:27:21,499] -[root] - [DEBUG] - Exploring with word 'bibliolater' current depth is '2'
[2021-06-23 15:27:21,528] -[root] - [DEBUG] - Exploring with word 'delivers' current depth is '2'
[2021-06-23 15:27:21,560] -[root] - [DEBUG] - Exploring with word 'support' current depth is '2'
[2021-06-23 15:27:21,592] -[root] - [DEBUG] - Exploring with word 'admit' current depth is '2'
[2021-06-23 15:27:21,622] -[root] - [DEBUG] - Exploring with word 'publication' current depth is '2'
[2021-06-23 15:27:21,652] -[root] - [DEBUG] - Exploring with word 'deem' current depth is '2'
[2021-06-23 15:27:21,689] -[root] - [DEBUG] - Exploring with word 'phonograph record' current depth is '2'
[2021-06-23 15:27:21,718] -[root] - [DEBUG] - Exploring with word 'stichometry' current depth is '2'
[2021-06

[2021-06-23 15:27:25,076] -[root] - [DEBUG] - Exploring with word 'library' current depth is '2'
[2021-06-23 15:27:25,132] -[root] - [DEBUG] - Exploring with word 'disk' current depth is '2'
[2021-06-23 15:27:25,188] -[root] - [DEBUG] - Exploring with word 'buch' current depth is '2'
[2021-06-23 15:27:25,245] -[root] - [DEBUG] - Exploring with word 'take for' current depth is '2'
[2021-06-23 15:27:25,302] -[root] - [DEBUG] - Exploring with word 'sacred scripture' current depth is '2'
[2021-06-23 15:27:25,359] -[root] - [DEBUG] - Exploring with word 'edition' current depth is '2'
[2021-06-23 15:27:25,417] -[root] - [DEBUG] - Exploring with word 'entertain' current depth is '2'
[2021-06-23 15:27:25,480] -[root] - [DEBUG] - Exploring with word 'concur' current depth is '2'
[2021-06-23 15:27:25,544] -[root] - [DEBUG] - Exploring with word 'take' current depth is '2'
[2021-06-23 15:27:25,610] -[root] - [DEBUG] - Exploring with word 'have got' current depth is '2'
[2021-06-23 15:27:25,677] -

[2021-06-23 15:27:27,771] -[root] - [DEBUG] - Exploring with word 'vol' current depth is '2'
[2021-06-23 15:27:27,807] -[root] - [DEBUG] - Exploring with word 'reading' current depth is '2'
[2021-06-23 15:27:27,846] -[root] - [DEBUG] - Exploring with word 'arrest' current depth is '2'
[2021-06-23 15:27:27,884] -[root] - [DEBUG] - Exploring with word 'interpretation' current depth is '2'
[2021-06-23 15:27:27,919] -[root] - [DEBUG] - Exploring with word 'include' current depth is '2'
[2021-06-23 15:27:27,957] -[root] - [DEBUG] - Exploring with word 'accuse' current depth is '2'
[2021-06-23 15:27:27,993] -[root] - [DEBUG] - Exploring with word 'arrange for' current depth is '2'
[2021-06-23 15:27:28,029] -[root] - [DEBUG] - Exploring with word 'album' current depth is '2'
[2021-06-23 15:27:28,068] -[root] - [DEBUG] - Exploring with word 'registration' current depth is '2'
[2021-06-23 15:27:28,107] -[root] - [DEBUG] - Exploring with word 'decide' current depth is '2'
[2021-06-23 15:27:28,14

[2021-06-23 15:27:30,930] -[root] - [INFO] - getting https://www.synonyms.com/synonym/read
[2021-06-23 15:27:30,932] -[urllib3.connectionpool] - [DEBUG] - Starting new HTTPS connection (1): www.synonyms.com:443
[2021-06-23 15:27:31,425] -[urllib3.connectionpool] - [DEBUG] - https://www.synonyms.com:443 "GET /synonym/read HTTP/1.1" 200 30093
[2021-06-23 15:27:31,598] -[root] - [INFO] - 196 found
[2021-06-23 15:27:31,604] -[root] - [DEBUG] - Exploring with word 'picture' current depth is '2'
[2021-06-23 15:27:31,617] -[root] - [DEBUG] - Exploring with word 'bear witness' current depth is '2'
[2021-06-23 15:27:31,632] -[root] - [DEBUG] - Exploring with word 'register' current depth is '2'
[2021-06-23 15:27:31,648] -[root] - [DEBUG] - Exploring with word 'pick up' current depth is '2'
[2021-06-23 15:27:31,661] -[root] - [DEBUG] - Exploring with word 'check' current depth is '2'
[2021-06-23 15:27:31,677] -[root] - [DEBUG] - Exploring with word 'train' current depth is '2'
[2021-06-23 15:27:

[2021-06-23 15:27:38,318] -[root] - [DEBUG] - Exploring with word 'maltese' current depth is '2'
[2021-06-23 15:27:38,384] -[root] - [DEBUG] - Exploring with word 'follows' current depth is '2'
[2021-06-23 15:27:38,448] -[root] - [DEBUG] - Exploring with word 'ingest' current depth is '2'
[2021-06-23 15:27:38,514] -[root] - [DEBUG] - Exploring with word 'lea' current depth is '2'
[2021-06-23 15:27:38,578] -[root] - [DEBUG] - Exploring with word 'enjoin' current depth is '2'
[2021-06-23 15:27:38,644] -[root] - [DEBUG] - Exploring with word 'hear' current depth is '2'
[2021-06-23 15:27:38,711] -[root] - [DEBUG] - Exploring with word 'meditate' current depth is '2'
[2021-06-23 15:27:38,777] -[root] - [DEBUG] - Exploring with word 'unravel' current depth is '2'
[2021-06-23 15:27:38,844] -[root] - [DEBUG] - Exploring with word 'receive' current depth is '2'
[2021-06-23 15:27:38,911] -[root] - [DEBUG] - Exploring with word 'be studious in books' current depth is '2'
[2021-06-23 15:27:38,979]

[2021-06-23 15:27:41,731] -[root] - [DEBUG] - Exploring with word 'decipher' current depth is '2'
[2021-06-23 15:27:41,760] -[root] - [DEBUG] - Exploring with word 'render' current depth is '2'
[2021-06-23 15:27:41,791] -[root] - [DEBUG] - Exploring with word 'point to' current depth is '2'
[2021-06-23 15:27:41,817] -[root] - [DEBUG] - Exploring with word 'puzzle out' current depth is '2'
[2021-06-23 15:27:41,849] -[root] - [DEBUG] - Exploring with word 'know' current depth is '2'
[2021-06-23 15:27:41,876] -[root] - [DEBUG] - Exploring with word 'telling' current depth is '2'
[2021-06-23 15:27:41,909] -[root] - [DEBUG] - Exploring with word 'stated' current depth is '2'
[2021-06-23 15:27:41,940] -[root] - [DEBUG] - Exploring with word 'inform' current depth is '2'
[2021-06-23 15:27:41,972] -[root] - [DEBUG] - Exploring with word 'peruse' current depth is '2'
[2021-06-23 15:27:42,003] -[root] - [DEBUG] - Exploring with word 'major in' current depth is '2'
[2021-06-23 15:27:42,033] -[roo

['PS',
 'accept',
 'accommodate',
 'according to the rules',
 'account book',
 'accountability',
 'accountancy',
 'accountant',
 'accounting',
 'accounts',
 'accuse',
 'acquire',
 'act',
 'adjudge',
 'admit',
 'adopt',
 'afl',
 'agree',
 'aim',
 "al-qur'an",
 'album',
 'allege',
 'allocate',
 'allow',
 'analyse',
 'analyze',
 'annuaire',
 'anthology',
 'appear in reading',
 'apply',
 'appropriate',
 'arrange',
 'arrange for',
 'arrest',
 'articulate',
 'ascertain',
 'ask',
 'assume',
 'augur',
 'aver',
 'bag',
 'balance sheet',
 'be read',
 'be studious in books',
 'bear',
 'bear witness',
 'bed',
 'bible',
 'biblioclast',
 'bibliognost',
 'bibliography',
 'bibliolater',
 'bibliomania',
 'bibliophilism',
 'bibliophobia',
 'bind',
 'biograph',
 'blame',
 'bollock',
 'bone up on',
 'book',
 'book',
 'book of account',
 'bookie',
 'booklet',
 'books',
 'bring',
 'browse',
 'buch',
 'bulk',
 'cahier',
 'call for',
 'canvas',
 'canvass',
 'carnet',
 'carry',
 'catalogue',
 'catch',
 'chaill

In [383]:
!python -m lexicons_builder parisien \
      --lang fr               \
      --out-file here.txt    \
      --depth 2             \
      --wolf-path wolf-1.0b4.xml \
      --web \
      --nlp-model frWac_postag_no_phrase_700_skip_cut50.bin

RDFLib Version: 5.0.0
[2021-06-23 21:22:46,220] -[rdflib] - [INFO] - RDFLib Version: 5.0.0
[2021-06-23 21:22:46,426] -[root] - [DEBUG] - user-Agent is 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36'
[2021-06-23 21:22:46,633] -[root] - [INFO] - exploring model 'frWac_postag_no_phrase_700_skip_cut50.bin' with word 'parisien' at 2 depth
[2021-06-23 21:22:46,633] -[root] - [DEBUG] - Exploring the model with word 'parisien' current depth is '1'
[2021-06-23 21:22:47,483] -[root] - [INFO] - loading nlp model from 'frWac_postag_no_phrase_700_skip_cut50.bin'
[2021-06-23 21:22:47,483] -[root] - [INFO] - Trying to load 'frWac_postag_no_phrase_700_skip_cut50.bin' with KeyedVectors binary=True
[2021-06-23 21:22:47,483] -[gensim.models.utils_any2vec] - [INFO] - loading projection weights from frWac_postag_no_phrase_700_skip_cut50.bin
[2021-06-23 21:22:47,483] -[smart_open.smart_open_lib] - [DEBUG] - {'transport_params': None, 'ignore_ext': Fal

In [20]:
933/len(communes2)

0.7518130539887188